# 라이브러리

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time

## Simple Text Crawling 
멜론 사이트에서 노래 제목을 크롤링해보자

URL: https://www.melon.com/chart/index.htm

In [3]:
DRIVER_PATH = 'chromedriver.exe'

In [6]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) 
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# title crawling
title = WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr:nth-child(1) > td:nth-child(4) > div > div")))

# print("Title: {}".format(title.text))

title.text

'Dynamite\n방탄소년단'

css selector의 규칙을 찾아본다
- 1번째 제목: #frm > div > table > tbody > tr:nth-child(1) > td:nth-child(4) > div > div
- 2번째 제목: #frm > div > table > tbody > tr:nth-child(2) > td:nth-child(4) > div > div
- 100번째 제목: #frm > div > table > tbody > tr:nth-child(100) > td:nth-child(4) > div > div
---
또는 XPATH로도 확인해보자 (full Xpath)
- 1번째 제목: //*[@id="frm"]/div/table/tbody/tr[1]/td[4]/div/div
- 2번째 제목: //*[@id="frm"]/div/table/tbody/tr[2]/td[4]/div/div


- 100번째 제목: //*[@id="frm"]/div/table/tbody/tr[100]/td[4]/div/div

In [8]:
# 5번째 제목 크롤링
WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.XPATH, "//*[@id='frm']/div/table/tbody/tr[5]/td[4]/div/div"))).text

'다시 여기 바닷가\n싹쓰리 (유두래곤, 린다G, 비룡)'

## Text Crawling with for loop

위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자.

In [9]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
for i in range(1, 51):
    title = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f"//*[@id='frm']/div/table/tbody/tr[{i}]/td[4]/div/div")))
    title_list.append(title.text)
    
print(title_list)

['Dynamite\n방탄소년단', '취기를 빌려 (취향저격 그녀 X 산들)\n산들', '눈누난나 (NUNU NANA)\n제시 (Jessi)', '마리아 (Maria)\n화사 (Hwa Sa)', '다시 여기 바닷가\n싹쓰리 (유두래곤, 린다G, 비룡)', 'When We Disco (Duet with 선미)\n박진영', '오래된 노래\n스탠딩 에그', 'How You Like That\nBLACKPINK', '에잇(Prod.&Feat. SUGA of BTS)\n아이유', '내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)\n규현 (KYUHYUN)', 'Dolphin\n오마이걸 (OH MY GIRL)', '아로하\n조정석', 'Downtown Baby\n블루 (BLOO)', '홀로\n이하이', 'Tight\n10CM', '어떻게 지내 (Prod. By VAN.C)\n오반', 'Summer Hate (Feat. 비)\n지코 (ZICO)', '덤디덤디 (DUMDi DUMDi)\n(여자)아이들', 'Blueming\n아이유', 'Dance Monkey\nTones And I', 'Not Shy\nITZY (있지)', '살짝 설렜어 (Nonstop)\n오마이걸 (OH MY GIRL)', 'Memories\nMaroon 5', '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야\n장범준', 'METEOR\n창모 (CHANGMO)', '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지\nAKMU (악동뮤지션)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)\n방탄소년단', 'Ice Cream (with Selena Gomez)\nBLACKPINK', '사랑은 지날수록 더욱 선명하게 남아\n전상근', '사랑하게 될 줄 알았어\n전미도', '그 여름을 틀어줘\n싹쓰리 (유두래곤, 린다G, 비룡)', '늦은 밤 너의 집 앞 골목길에서\n노을', '거짓말이라도 해서 널 보고싶어\n백지영', '보라빛 밤 (pporappippam)\n선미', '모든 날, 모

나중에 필요한 변수(제목, 가수, 가사... 등)을 모두 긁어 한번에 데이터프레임으로 저장하여 보관한다!

## Text Crawling (Click & Back)
클릭하고 나오기 --> 동적 크롤링 가능 (가사 크롤링 가능)

노래 제목에 링크가 걸려있기 때문에, 해당 링크까지의 XPath를 추가한다.

In [11]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/a')))
click_element.click()    

# back
driver.back()


# 2번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[2]/td[3]/div/a')))
click_element.click()    

# back
driver.back()

## Text Crawling including content
- 1.2처럼 for문과 함께 써보자! (첫 페이지 5개의 글에 대해 title, artist, heart(하트 갯수), lyrics(가사)를 크롤링
- 1.3에서 사용한 click & back을 활용하자

In [14]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
artist_list = []
heart_list = []
lyrics_list = []

# crawling (TOP 5)
for i in range(1, 6):
    # click
    click_element = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{i}]/td[3]/div/a')))
    click_element.click()

    # title crawling --> 노래 제목 가져옴 
    title = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))
    title_list.append(title.text)

    # artist crawling --> 아티스트 이름 가져오기
    artist = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
    artist_list.append(artist.text)
    
    # heart crawling --> 좋아요 수 가져오기 밑의 css selctor를 확인해보자.
    heart = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
    heart_list.append(heart.text)

    # lyrics crawling
    lyrics = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
    lyrics_list.append(lyrics.text)
    
    # back
    driver.back()
    
print(title_list)
print(artist_list)
print(heart_list)
print(lyrics_list)

['Dynamite', '취기를 빌려 (취향저격 그녀 X 산들)', '눈누난나 (NUNU NANA)', '마리아 (Maria)', '다시 여기 바닷가']
['방탄소년단', '산들', '제시 (Jessi)', '화사 (Hwa Sa)', '싹쓰리 (유두래곤, 린다G, 비룡)']
['273,098', '99,009', '74,963', '139,016', '221,791']
["Cos ah ah\nI’m in the stars tonight\nSo watch me bring the fire\nand set the night alight\nShoes on get up in the morn\nCup of milk let’s rock and roll\nKing Kong kick the drum\nrolling on like a rolling stone\nSing song when I’m walking home\nJump up to the top LeBron\nDing dong call me on my phone\nIce tea and a game of ping pong\nThis is getting heavy\nCan you hear the bass boom\nI’m ready\nLife is sweet as honey\nYeah this beat cha ching\nlike money\nDisco overload I’m into\nthat I’m good to go\nI'm diamond you know I glow up\nHey so let’s go\nCos ah ah\nI’m in the stars tonight\nSo watch me bring the fire\nand set the night alight\nShining through the city\nwith a little funk and soul\nSo I’mma light it up\nlike dynamite woah\nBring a friend join the crowd\nWhoever wanna com

### TIP: 보통 결과값을 데이터프레임 형태로 저장한다.

In [15]:
# 결과 변수 --> 결과변수를 일단 딕셔너리 형태로 저장한 후, 이거를 pandas 모듈을 이용해 데이터프레임으로 변환시킨다.
raw_result = {'title': title_list,
              'artist': artist_list,
              'heart': heart_list,
          'lyrics': lyrics_list}

result = pd.DataFrame(raw_result)

# # csv 파일로 save
# result.to_csv("MelonTop5", mode='w') --> MelonTop5.csv로 현재 디렉토리에 저장 

# driver 종료
driver.quit()

In [16]:
result

title               artist    heart  \
0               Dynamite                방탄소년단  273,098   
1  취기를 빌려 (취향저격 그녀 X 산들)                   산들   99,009   
2       눈누난나 (NUNU NANA)           제시 (Jessi)   74,963   
3            마리아 (Maria)          화사 (Hwa Sa)  139,016   
4              다시 여기 바닷가  싹쓰리 (유두래곤, 린다G, 비룡)  221,791   

                                              lyrics  
0  Cos ah ah\nI’m in the stars tonight\nSo watch ...  
1  언제부턴가 불쑥\n내 습관이 돼버린 너\n혹시나 이런 맘이\n어쩌면 부담일까\n널 ...  
2  I’m trying to give u\nsomething more\nSo come ...  
3  욕을 하도 먹어 체했어 하도\n서러워도 어쩌겠어 I do\n모두들 미워하느라 애썼네...  
4  예아 호우 예예예\n싹쓰리 인더 하우스\n커커커커커몬 싹 쓰리 투 렛츠고\n나 다시...

## Image Crawling
이미지 크롤링하기 # 예시 셀렉터가 변경된 것으로 보인다.

- 1번째 이미지: #frm > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > div > a > img
- 2번째 이미지: #frm > div > table > tbody > tr:nth-child(2) > td:nth-child(2) > div > a > img


- 50번째 이미지: #frm > div > table > tbody > tr:nth-child(50) > td:nth-child(2) > div > a > img


#### STEP1.URL Craling

In [17]:
#  
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
link_list = []

# # img crawling (TOP 50)
for i in range(1, 51):
    
    img = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(2) > div > a > img")))

    link_list.append(img.get_attribute('src'))

print(link_list)

['https://cdnimg.melon.co.kr/cm2/album/images/104/79/150/10479150_20200918102847_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/63/600/10463600_20200720152905_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/69/416/10469416_20200730151034_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/52/351/10452351_20200629152036_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/62/799/10462799_20200717150822_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/75/061/10475061_20200812120927_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm/album/images/021/48/596/2148596_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/51/566/10451566_20200626114914_500.jpg/melon/resize/120/quality/80/optimize', 

#### STEP2. Download images using URLs
자신의 디렉토리에 img 폴더를 생성하고 실행

In [20]:
import urllib.request

count = 0
for link in link_list:
    count += 1
    urllib.request.urlretrieve(link, './img/img' + str(count) + '.jpg') # img 폴더를 직접 만들었따.